In [1]:
import pandas as pd
import numpy as np

In [9]:
def state_propagation(q0, prob_transition_matrix, n_steps):
    q = q0.copy()
    for _ in range(n_steps):
        q = M @ q
    return q

In [23]:
# 1 - state propagation

M = pd.read_csv('M.csv', header=None)
t = 1024
q0 = [1,0,0,0,0,0,0,0,0,0]

state_propagation(q0, M, t)

0    0.051031
1    0.043740
2    0.128057
3    0.186130
4    0.087481
5    0.117262
6    0.088499
7    0.083189
8    0.117262
9    0.097349
dtype: float64

In [57]:
# 1 - random walk
def random_walk(q0, prob_transition_matrix, n_burn_in_steps, n_steps_after_burn_in):
    #q0 should be all 0s except one 1
    M = prob_transition_matrix.copy()
    i = q0.index(1)
    node_options = np.arange(M.shape[0])
    
    for _ in range(n_burn_in_steps):
        i = np.random.choice(node_options, p=M[i].to_numpy())

    num_times_at_each_node = np.zeros(M.shape[0])
    for _ in range(n_steps_after_burn_in):
        i = np.random.choice(node_options, p=M[i].to_numpy())
        num_times_at_each_node[i] += 1

    q_star = num_times_at_each_node / num_times_at_each_node.sum()
    return q_star

In [58]:
M = pd.read_csv('M.csv', header=None)
t = 1024
q0 = [1,0,0,0,0,0,0,0,0,0]
burn_in = 100

print(random_walk(q0, M, burn_in, t))

[0.09277344 0.08300781 0.15039062 0.22460938 0.09765625 0.11914062
 0.05371094 0.04492188 0.07714844 0.05664062]
